In [3]:
# --------------------------------------------------
# Darie-Dragos Mitoiu
# Coursework Part B v1.0.0 25/11/2019
# Big Data Analysis Coursework Part B
# --------------------------------------------------

In [4]:
# Initial Exploration

# Declare medallion data path
medallion_data_path = "/CW2019/Medallion.csv"

In [5]:
# Declare livery data path
livery_data_path = "/CW2019/Livery.csv"

In [6]:
# Load medallion data
medallion_data = sc.textFile(medallion_data_path)

In [7]:
# Load livery data
livery_data = sc.textFile(livery_data_path)

In [8]:
# Show first 5 lines of original medallion data
medallion_data.take(5)

[u'License Number;Name;Type;Expiration Date;Last Date Updated;Last Time Updated',
 u'679504;POTTER,WILLIAM,C;MEDALLION TAXI DRIVER;03/28/2020;08/22/2019;13:20',
 u'694722;KATTARIA,ABDEL,W;MEDALLION TAXI DRIVER;03/22/2020;08/22/2019;13:20',
 u'694770;MEDINA,FRANKLIN,A;MEDALLION TAXI DRIVER;03/24/2020;08/22/2019;13:20',
 u'503369;SHOHAG,MAHMADUL;MEDALLION TAXI DRIVER;03/05/2021;08/22/2019;13:20']

In [9]:
# Show first 5 lines of original livery data
livery_data.take(5)

[u'License Number;Name;Status Code;Status Description;Expiration Date;Last Date Updated;Last Time Updated',
 u'493092;MAIGA,BOUNAFOU;1;SHL UNRESTRICTED;11/24/2019;08/22/2019;13:20',
 u'681774;GNANARAJ,SINNA;1;SHL UNRESTRICTED;04/03/2020;08/22/2019;13:20',
 u'498143;KAREEM,LUKMAN,ABDUL;1;SHL UNRESTRICTED;01/03/2021;08/22/2019;13:20',
 u'686178;TAVERAS,OLGA;1;SHL UNRESTRICTED;08/21/2022;08/22/2019;13:20']

In [10]:
# Remove header of medallion data

# Get medallion data header
medallion_header = medallion_data.first()
# Remove medallion data headers
rm_medallion_data = medallion_data.filter(lambda line: medallion_header not in line)
# Show first 5 lines
rm_medallion_data.take(5)

[u'679504;POTTER,WILLIAM,C;MEDALLION TAXI DRIVER;03/28/2020;08/22/2019;13:20',
 u'694722;KATTARIA,ABDEL,W;MEDALLION TAXI DRIVER;03/22/2020;08/22/2019;13:20',
 u'694770;MEDINA,FRANKLIN,A;MEDALLION TAXI DRIVER;03/24/2020;08/22/2019;13:20',
 u'503369;SHOHAG,MAHMADUL;MEDALLION TAXI DRIVER;03/05/2021;08/22/2019;13:20',
 u'695848;RODRIGUEZ,JOSE,E;MEDALLION TAXI DRIVER;03/13/2020;08/22/2019;13:20']

In [11]:
# Number of drivers in the original medallion data

# Count number of rows of original medallion data
print "Number of rows in the original medallion data:", medallion_data.count()

Number of rows in the original medallion data: 185416


In [12]:
# Number of drivers in the medallion data after the deletion of header

# Count number of rows of medallion data without the headers
print "Number of rows in the medallion data after the remove of headers:", rm_medallion_data.count()

Number of rows in the medallion data after the remove of headers: 185415


In [13]:
# Remove header of livery data

# Get the headers line
livery_header = livery_data.first()
# Remove the header line
rm_livery_data = livery_data.filter(lambda line: livery_header not in line)
# Show the first 5 lines
rm_livery_data.take(5)

[u'493092;MAIGA,BOUNAFOU;1;SHL UNRESTRICTED;11/24/2019;08/22/2019;13:20',
 u'681774;GNANARAJ,SINNA;1;SHL UNRESTRICTED;04/03/2020;08/22/2019;13:20',
 u'498143;KAREEM,LUKMAN,ABDUL;1;SHL UNRESTRICTED;01/03/2021;08/22/2019;13:20',
 u'686178;TAVERAS,OLGA;1;SHL UNRESTRICTED;08/21/2022;08/22/2019;13:20',
 u'664245;CORNIEL,PEDRO,A;1;SHL UNRESTRICTED;04/10/2022;08/22/2019;13:20']

In [14]:
# Number of drivers in the original livery data

# Count number of rows in the original livery data
print "Number of rows in the original livery data:", livery_data.count()

Number of rows in the original livery data: 186739


In [15]:
# Number of drivers in the livery data after the deletion of the header

# Count number of rows in the livery data after the deletion of header
print "Number of rows in the livery data after the deletion of header:", rm_livery_data.count()

Number of rows in the livery data after the deletion of header: 186738


In [24]:
# Number of values of status description

count_status = rm_livery_data.map(lambda line: line.split(";")) \
                             .map(lambda fields: (fields[3], 1)) \
                             .reduceByKey(lambda v1,v2: v1+v2)

count_status.collect()


[(u'SHL UNRESTRICTED', 185359),
 (u'SHL WAV ONLY    ', 1323),
 (u'SHL NON-WAV ONLY', 56)]

In [31]:
# Get the status description values
values_status = count_status.map(lambda fields: (fields[0]))

# Count values and print result
print "Values of status description:", values_status.count()

Values of status description: 3


In [32]:
# Total number of values of status description

# Get count column
count_status_result = count_status.map(lambda fields: (fields[1]))

# Print result
print "Total number of status description values:", count_status_result.sum()

Total number of status description values: 186738


In [36]:
# More Detailed Analysis - A

# Get the license number and status description for join later on
drivers_status = rm_livery_data.map(lambda line: line.split(";")) \
                               .map(lambda fields: (fields[0], fields[3]))

In [37]:
# Show first 5 lines
drivers_status.take(5)

[(u'493092', u'SHL UNRESTRICTED'),
 (u'681774', u'SHL UNRESTRICTED'),
 (u'498143', u'SHL UNRESTRICTED'),
 (u'686178', u'SHL UNRESTRICTED'),
 (u'664245', u'SHL UNRESTRICTED')]

In [38]:
# Create medallion data key and value pair
medallion_pair = rm_medallion_data.map(lambda line: line.split(";")) \
                                  .map(lambda fields: (fields[0],  fields[1:]))
    
# Show first 5 lines
medallion_pair.take(5)

[(u'679504',
  [u'POTTER,WILLIAM,C',
   u'MEDALLION TAXI DRIVER',
   u'03/28/2020',
   u'08/22/2019',
   u'13:20']),
 (u'694722',
  [u'KATTARIA,ABDEL,W',
   u'MEDALLION TAXI DRIVER',
   u'03/22/2020',
   u'08/22/2019',
   u'13:20']),
 (u'694770',
  [u'MEDINA,FRANKLIN,A',
   u'MEDALLION TAXI DRIVER',
   u'03/24/2020',
   u'08/22/2019',
   u'13:20']),
 (u'503369',
  [u'SHOHAG,MAHMADUL',
   u'MEDALLION TAXI DRIVER',
   u'03/05/2021',
   u'08/22/2019',
   u'13:20']),
 (u'695848',
  [u'RODRIGUEZ,JOSE,E',
   u'MEDALLION TAXI DRIVER',
   u'03/13/2020',
   u'08/22/2019',
   u'13:20'])]

In [39]:
# More Detailed Analysis - B

# Join medallion data pair with livery licencese number and status description
medallion_status = medallion_pair.join(drivers_status)

In [40]:
# Show first 5 lines
medallion_status.take(5)

[(u'5866631',
  ([u'AVALBOEV,JURABEK',
    u'MEDALLION TAXI DRIVER',
    u'04/23/2021',
    u'08/22/2019',
    u'13:20'],
   u'SHL UNRESTRICTED')),
 (u'5866635',
  ([u'DZAGNIDZE,ZURABI',
    u'MEDALLION TAXI DRIVER',
    u'04/24/2021',
    u'08/22/2019',
    u'13:20'],
   u'SHL UNRESTRICTED')),
 (u'5441506',
  ([u'ZARIPOV,BAHODIR',
    u'MEDALLION TAXI DRIVER',
    u'06/28/2020',
    u'08/22/2019',
    u'13:20'],
   u'SHL UNRESTRICTED')),
 (u'5333263',
  ([u'SINGH,SEWAK',
    u'MEDALLION TAXI DRIVER',
    u'05/28/2022',
    u'08/22/2019',
    u'13:20'],
   u'SHL UNRESTRICTED')),
 (u'5839477',
  ([u'AGBAMA,JAMES',
    u'MEDALLION TAXI DRIVER',
    u'12/27/2020',
    u'08/22/2019',
    u'13:20'],
   u'SHL UNRESTRICTED'))]

In [47]:
# More Detailed Analysis - C

# Inverse status description and key
status = medallion_status.map(lambda fields: (fields[1][1], fields[0]))
# Count the status description
status_count = status.map(lambda fields: (fields[0], 1)) \
                     .reduceByKey(lambda v1, v2: v1+v2)
# Show the result
status_count.collect()

[(u'SHL UNRESTRICTED', 185359), (u'SHL NON-WAV ONLY', 56)]

In [48]:
# Get the status description numbers
number_drivers_status = status_count.map(lambda fields: (fields[1]))
# Calculate the total number
print "Number of drivers having each status description:", number_drivers_status.sum()

Number of drivers having each status description: 185415


In [49]:
# More Detailed Analysis - D

# Get the last name
d_medallion = rm_medallion_data.map(lambda line: line.split(";")[1])

# Frequency of the names in descending order
d_medallion_name = d_medallion.map(lambda line: line.split(",")[0]) \
                              .map(lambda name: (name, 1)) \
                              .reduceByKey(lambda v1,v2: v1+v2) \
                              .map(lambda fields: (fields[1], fields[0])) \
                              .sortByKey(ascending=False) \
                              .map(lambda fields: (fields[1], fields[0]))

# Show first 5 lines
d_medallion_name.take(5)

[(u'SINGH', 7459),
 (u'AHMED', 2179),
 (u'KHAN', 1840),
 (u'ISLAM', 1832),
 (u'RAHMAN', 1779)]